# Assignment 2: Hypothesis Testing (Solutions)

*Due*: Wed, Nov 13, 2019 in class

*Submission*: Complete this notebook and print out the output or electronically submit it. 

Everything you need to complete is marked with a TODO. For textual questions create a new cell under the question to respond to it.


## Motivation
In a standard randomized control trial, our null hypothesis is often trivial---nothing happens, no difference in the mean, no difference in the relative ranking. In this assignment, we look to generalize this idea to compare observed data against an assumed statistical model. That is, could the observed data plausibly be generated from the known model.


An air shower is a cascade of ionized particles and electromagnetic radiation produced in the atmosphere when a primary cosmic ray (i.e. one of extraterrestrial origin) enters the atmosphere. When a particle, which could be a proton, a nucleus, an electron, a photon, or (rarely) a positron, strikes an atom's nucleus in the air it produces many energetic hadrons. We have a detector that observes particles that reach a ground station and measures the particle energy and arrival time.

## Model and Data
We have the following theoretical model to describe particle behavior. The energy of each particle is drawn independent of arrival time from a [Gamma Distribution](https://en.wikipedia.org/wiki/Gamma_distribution). The particles arrive as a [Poisson process](https://en.wikipedia.org/wiki/Poisson_point_process). We have the following simulator:

In [3]:
import numpy as np
import pandas as pd

def simulate_burst(total):
    """Simulates a trial of total # of particles returns
       a dataframe with two columns one with observed time (in microsecs)
       and one with the energy in kilojoules.
    """
    t = 0
    data = []
    for trial in range(total):
        t += np.random.exponential(scale=1.0)
        obs = np.random.gamma(2.15,1.96, 1)[0]
        
        data.append({'otime_us':t, 'energy_kj':obs})
    
    return pd.DataFrame(data)

In addition to the simulator, you are given a daset of real observations [download](https://raw.githubusercontent.com/sjyk/cmsc21800/master/part.csv). You will write a function to load this dataset into a pandas dataframe. The dataset contains some missing values, the function should drop all rows with any missing values (i.e., NaN)

In [19]:
def load_data(filename):
    """Input: a csv file of airburst observations
       Output: a pandas dataframe with no NaNs
    """
    
    df = pd.read_csv(filename, delimiter = ' ')#note the delimeter is a space
    return df[~df['energy_kj'].isna()]

## Pre-Processing

Before, we begin testing, we show analyze the data for potential problems.

Q1. TODO *Compare particle energies generated from the simulator and the real data. If they do differ, explain how.*

In [25]:
simulate_burst(1000).describe()

,otime_us,energy_kj
count,1000.000000,1000.000000
mean,496.292091,4.105662
std,283.243018,2.752229
min,1.891547,0.178888
25%,248.046612,2.081817
50%,485.571582,3.532685
75%,751.013908,5.602809
max,988.634117,20.897226


In [27]:
load_data('part.csv').describe() #the means are very similar but the spread (std, max value, min) are very different

,otime_us,energy_kj
count,996.000000,996.000000
mean,498.045568,4.385513
std,287.746567,33.995733
min,0.940633,-246.000000
25%,249.466874,1.940490
50%,497.730739,3.559943
75%,746.774461,5.958558
max,998.868280,286.000000


Q2. TODO *Your engineers tell you that all energy readings should be positive. Are there any negative values in either of the datasets? If so, is there any unexpected pattern to those values in terms of times they occur or values they take on?*

In [28]:
df = load_data('part.csv') #they are all integer values.
df[df['energy_kj'] < 0]

,otime_us,energy_kj
1,1.559228,-128.0
60,60.057136,-139.0
78,78.842368,-104.0
90,90.386515,-24.0
91,91.418192,-68.0
97,97.516804,-89.0
99,99.581593,-85.0
109,109.091903,-32.0
139,139.651419,-3.0
140,140.264270,-96.0


Q3. TODO *Are there any other energy readings that are suspect in the real dataset? Roughly what fraction of values are suspect*

In [30]:
df[np.round(df['energy_kj']) == df['energy_kj']] #all the large outliers are integers

,otime_us,energy_kj
1,1.559228,-128.0
10,10.328715,179.0
29,29.357244,286.0
40,40.450206,77.0
60,60.057136,-139.0
...,...,...
921,921.007503,94.0
930,930.771796,41.0
953,953.112440,-113.0
958,958.856752,47.0


Based on your answers to Q1, Q2, Q3, write a function that cleans the real data by removing all problematic observations. 

In [66]:
def clean(df):
    """
       Input: a dataframe of a mix of erroneous and correct observations 
       Output: a dataframe with only the correct observations
    """
    
    return df[(df['energy_kj'] > 0) & (df['energy_kj'] < 20)]

## Comparing the Energies
Now, we will compare the particle energies from the simulated data and the real data. Fill in the following hypothesis tests. *Be reasonable about this* You may not import methods from statistics packages that perform the test for you.

In [71]:
from scipy.stats import *

def z_test_energy(simulated, real):
    """Input: a dataframe of simulated observations, a dataframe of real observations 
       Output: a p-value based on the two-sample z-test
    """
    
    diff = np.abs(np.mean(simulated['energy_kj']) - np.mean(real['energy_kj']))
    se = np.sqrt(sem(simulated['energy_kj'])**2 + sem(real['energy_kj'])**2 )
    return ((1-norm.cdf(diff/se))
    
    

#get the data
simulated = simulate_burst(10000)
real =  clean(load_data('part.csv'))

#two acceptable solutions here 1-tailed or 2-tailed
print('Two-Sample Z-Test: ', z_test_energy(simulated, real)) #not significant > 0.1 full credit

#print('Rank Sum Test: ', rs_test_energy(simulated, real)) #significant < 0.05 full credit

Two-Sample Z-Test:  (0.8890915288951455, 0.44454576444757277)
Rank Sum Test:  (0.0, 0.0)


There are a few parameters here that we selected: (1) how much data to simulate, and (2) we cleaned the real dataset prior to testing. Evaluate the effects of these choices.

Q1. TODO *Would it be beneficial to simulate 1e6 data points rather than the 10000 used above, why or why not?*

In [ ]:
#No it would not be. The issue is with the null hypothesis not the data.

Q2. TODO *Do the p-values change if you did not clean the dataset? Are both tests equally sensitive to the dirty data?*

In [ ]:
#yes it does. The ztest is particularly sensitive

## Comparing the Arrival Times
So far, we have only tested the particle energies. Another important aspect of our model is the arrival process (i.e., the times). 

Q1. TODO *Describe a hypothesis test that evaluates whether the arrival process significantly differs in the simulator from the observed data.*

Check the intervals

Q2. TODO *Do your pre-processing choice above change? Why or why not?*

In [73]:
def test_arrival_process(simulated, real):
    """Input: a dataframe of simulated observations, a dataframe of real observations 
       Output: a p-value that determines the difference between the arrival processes
    """
    
    u = np.array(simulated['otime_us'])
    udiff = u[1:] - u[:-1] 
    
    v = np.array(real['otime_us'])
    vdiff = v[1:] - v[:-1] 

    diff = np.abs(np.mean(u) - np.mean(v))
    se = np.sqrt(sem(u)**2 + sem(v)**2 )
    return (1-norm.cdf(diff/se)

test_arrival_process(simulated, real) #significant!!

(0.0, 0.0)